#CNN+LSTM Model

In [1]:
from google.colab import files
uploaded = files.upload()

Saving resampled_abnormal.npy to resampled_abnormal.npy
Saving resampled_normal.npy to resampled_normal.npy


In [2]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam

# 1. 데이터 불러오기
X_normal = np.load('resampled_normal.npy')      # shape (n, 100)
X_abnormal = np.load('resampled_abnormal.npy')  # shape (m, 100)

y_normal = np.zeros(len(X_normal), dtype=int)
y_abnormal = np.ones(len(X_abnormal), dtype=int)

X = np.concatenate([X_normal, X_abnormal], axis=0)
y = np.concatenate([y_normal, y_abnormal], axis=0)

# 2. 입력 차원 맞추기: (샘플 수, 시퀀스 길이, 채널 수)
X = X.reshape(-1, 100, 1)

# 3. K-Fold로 학습
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold = 1
accuracies = []

for train_idx, val_idx in kf.split(X, y):
    print(f"\n📂 Fold {fold}")

    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # 4. CNN + LSTM 모델 정의
    input_layer = Input(shape=(100, 1))

    x = Conv1D(32, kernel_size=5, activation='relu')(input_layer)
    x = MaxPooling1D(pool_size=2)(x)

    x = Conv1D(64, kernel_size=3, activation='relu')(x)
    x = MaxPooling1D(pool_size=2)(x)

    x = LSTM(64, return_sequences=False)(x)  # LSTM: 시간 정보 학습
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=input_layer, outputs=output)
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

    # 5. 학습
    model.fit(X_train, y_train, epochs=15, batch_size=32, verbose=0)

    # 6. 평가
    y_pred = (model.predict(X_val) > 0.5).astype("int32").flatten()
    acc = accuracy_score(y_val, y_pred)
    print(f"✅ Fold {fold} Accuracy: {acc:.4f}")
    accuracies.append(acc)
    fold += 1

# 7. 평균 정확도 출력
print(f"\n📊 평균 정확도 (5-Fold): {np.mean(accuracies):.4f}")


📂 Fold 1
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
✅ Fold 1 Accuracy: 0.9867

📂 Fold 2
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
✅ Fold 2 Accuracy: 0.9897

📂 Fold 3
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
✅ Fold 3 Accuracy: 0.9860

📂 Fold 4
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
✅ Fold 4 Accuracy: 0.9808

📂 Fold 5
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
✅ Fold 5 Accuracy: 0.9867

📊 평균 정확도 (5-Fold): 0.9860
